In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import census_key
import json

In [2]:
# year_list = ['2009', '2010','2011', '2012', '2013', '2014', '2015', '2016']

year_list = ['2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
fail_master = []

fuck_you_census = {
    '2009': '',
    '2010': 'acs/',
    '2011': 'acs/',
    '2012': 'acs/',
    '2013': 'acs/',
    '2014': 'acs/',
    '2015': 'acs/',
    '2016': 'acs/'
}


variable_conversion = {
    '2009': ['B19013_001E', 'B25064_001E'],
    '2010': ['B19013_001E', 'B25064_001E'],
    '2011': ['B19013_001E', 'B25064_001E'],
    '2012': ['B19013_001E', 'B25064_001E'],
    '2013': ['B19013_001E', 'B25064_001E'],
    '2014': ['B19013_001E', 'B25064_001E'],
    '2015': ['B19013_001E', 'B25031_001E'],
    '2016': ['B19013_001E', 'B25031_001E']
}

In [17]:
for year in year_list:
    try:
        census_test = requests.get(f'https://api.census.gov/data/{year}/acs/flows?get=FULL1_NAME,FULL2_NAME,MOVEDIN,MOVEDOUT,MOVEDNET' +
                                       f'&for=county:*&in=state:06&key=' + census_key).json()
        census_df = pd.DataFrame(census_test)
        census_df.columns = census_df.iloc[0]
        census_df = census_df[1:]
        census_df = census_df.rename(columns = {'FULL1_NAME': 'County',
                                                'FULL2_NAME': 'Origin',
                                                'MOVEDIN': f'{year}IN',
                                                'MOVEDOUT': f'{year}OUT',
                                                'MOVEDNET': f'{year}NET'
                                                })
        census_df = census_df[['County', 'Origin', f'{year}IN', f'{year}OUT', f'{year}NET']]
        
        economic_test = requests.get(f'https://api.census.gov/data/{year}/{fuck_you_census[year]}acs5?get=NAME,' + 
                                     f'{variable_conversion[year][0]},' +
                                     f'{variable_conversion[year][1]}' +
                                        '&for=county:*&in=state:06&key=' +
                                        census_key).json()
        economic_df = pd.DataFrame(economic_test)
        economic_df.columns = economic_df.iloc[0]
        economic_df = economic_df[1:]
        economic_df = economic_df.rename(columns = {f'{variable_conversion[year][0]}': f'{year}MED_INC',
                                                f'{variable_conversion[year][1]}': f'{year}RENT'
                                                })
        economic_df = economic_df[['NAME', f'{year}MED_INC', f'{year}RENT']]
        
        if year == '2016':
            census_master = census_df
            economic_master = economic_df
            
        else:
            census_master = census_master.merge(census_df, on = ['County', 'Origin'], how = 'left')
            economic_master = economic_master.merge(economic_df, on = 'NAME')
    
    except:
        fail_master.append(year)
        print(year+" FAILED")

In [4]:
census_master.to_csv('census_master.csv')

In [13]:
census_merged = pd.read_csv('census_master.csv')

In [14]:
census_merged.fillna(0)
census_merged = census_merged.drop(["Unnamed: 0", 'Origin'],axis=1)

In [15]:
census_group = census_merged.groupby('County').sum()
census_group = census_group.reset_index()

In [16]:
census_group['County'] = census_group['County'].str.replace(r' .*', '')

In [9]:
economic_master['NAME'] = economic_master['NAME'].str.replace(r' .*', '')

In [23]:
census_group

,County,2016IN,2016OUT,2016NET,2015IN,2015OUT,2015NET,2014IN,2014OUT,2014NET,...,2012NET,2011IN,2011OUT,2011NET,2010IN,2010OUT,2010NET,2009IN,2009OUT,2009NET
0,Alameda County,113013,93326.0,-167.0,108017.0,89267.0,960.0,103183.0,87176.0,-481.0,...,-1464.0,91626.0,81142.0,-4627.0,96455.0,87448.0,-6756.0,97434.0,86874.0,-4715.0
1,Alpine County,160,110.0,42.0,183.0,253.0,-82.0,165.0,267.0,-102.0,...,-86.0,52.0,364.0,-312.0,60.0,166.0,-106.0,69.0,146.0,-77.0
2,Amador County,3724,2868.0,806.0,3493.0,2700.0,724.0,2896.0,2203.0,649.0,...,18.0,2189.0,2351.0,-202.0,2612.0,2464.0,108.0,3052.0,2571.0,445.0
3,Butte County,14982,12840.0,1433.0,13463.0,12349.0,339.0,13296.0,12073.0,469.0,...,282.0,12867.0,11260.0,634.0,13958.0,10956.0,2109.0,15418.0,11097.0,3375.0
4,Calaveras County,2393,2583.0,-243.0,2148.0,2452.0,-353.0,2023.0,2087.0,-141.0,...,510.0,1848.0,1468.0,341.0,1922.0,2000.0,-131.0,2034.0,2115.0,-160.0
5,Colusa County,698,1445.0,-807.0,635.0,1266.0,-717.0,576.0,1227.0,-839.0,...,-551.0,647.0,814.0,-374.0,1119.0,888.0,-144.0,1376.0,1265.0,-426.0
6,Contra Costa County,66721,49837.0,8139.0,63410.0,46755.0,7910.0,61158.0,45507.0,7366.0,...,3133.0,54595.0,45786.0,1742.0,53893.0,48763.0,-2585.0,53860.0,50122.0,-3516.0
7,Del Norte County,2538,2431.0,80.0,2806.0,2631.0,132.0,2863.0,2210.0,613.0,...,676.0,2645.0,2025.0,596.0,2697.0,1731.0,902.0,2244.0,1695.0,493.0
8,El Dorado County,11187,11786.0,-1084.0,9543.0,10904.0,-1985.0,8899.0,10054.0,-1796.0,...,-1164.0,9023.0,8766.0,-263.0,9352.0,8512.0,356.0,10342.0,9000.0,692.0
9,Fresno County,35098,31067.0,-438.0,35814.0,31075.0,598.0,32931.0,30093.0,-1435.0,...,-1210.0,30280.0,27662.0,-1856.0,29662.0,28057.0,-2975.0,31770.0,28814.0,-1793.0


In [10]:
census_final = census_group.merge(economic_master, left_on = 'County', right_on = 'NAME')

In [11]:
census_final

,County,2016IN,2016OUT,2016NET,2015IN,2015OUT,2015NET,2014IN,2014OUT,2014NET,...,2013MED_INC,2013RENT,2012MED_INC,2012RENT,2011MED_INC,2011RENT,2010MED_INC,2010RENT,2009MED_INC,2009RENT
0,Alameda County,113013,93326.0,-167.0,108017.0,89267.0,960.0,103183.0,87176.0,-481.0,...,72112,1289,71516,1265,70821,1228,69384,1188,68863,1166
1,Alpine County,160,110.0,42.0,183.0,253.0,-82.0,165.0,267.0,-102.0,...,58636,1018,59931,755,59018,796,63478,913,54605,820
2,Amador County,3724,2868.0,806.0,3493.0,2700.0,724.0,2896.0,2203.0,649.0,...,53684,1006,53462,1039,56180,1011,54758,1059,57347,1075
3,Butte County,14982,12840.0,1433.0,13463.0,12349.0,339.0,13296.0,12073.0,469.0,...,43752,890,43339,885,42971,878,43170,861,42002,843
4,Calaveras County,2393,2583.0,-243.0,2148.0,2452.0,-353.0,2023.0,2087.0,-141.0,...,55295,1061,54686,1043,55256,1031,54971,948,57676,920
5,Colusa County,698,1445.0,-807.0,635.0,1266.0,-717.0,576.0,1227.0,-839.0,...,52158,923,52165,904,49558,885,48016,830,47150,762
6,Contra Costa County,66721,49837.0,8139.0,63410.0,46755.0,7910.0,61158.0,45507.0,7366.0,...,78756,1365,78187,1340,79135,1309,78385,1270,77838,1239
7,Del Norte County,2538,2431.0,80.0,2806.0,2631.0,132.0,2863.0,2210.0,613.0,...,37909,914,39626,865,37588,869,36118,845,38408,770
8,El Dorado County,11187,11786.0,-1084.0,9543.0,10904.0,-1985.0,8899.0,10054.0,-1796.0,...,69297,1090,70117,1109,68815,1098,70000,1074,70765,1062
9,Fresno County,35098,31067.0,-438.0,35814.0,31075.0,598.0,32931.0,30093.0,-1435.0,...,45563,875,45741,864,46903,850,46430,819,46230,801


In [12]:
census_final.to_csv('census_final.csv')